<a href="https://colab.research.google.com/github/camipower/BERT/blob/main/BERT_NER_TensorFlow_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install huggingface transformers

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 33.8MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 


### Check GPU Availability

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import tensorflow as tf
print(tf.version.VERSION)

2.5.0


### Install TensorFlow Model Garden

In [4]:
#!git clone --depth 1 -b v2.5.0 https://github.com/tensorflow/models.git

In [5]:
# install requirements to use tensorflow/models repository (restart runtime afterwards)
#!pip install -Uqr models/official/requirements.txt

### Import Dataset

In [4]:
import re
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import sys, os
#sys.path.append('models')
#from official.nlp.data import classifier_data_lib
#from official.nlp.bert import tokenization
#from official.nlp import optimization

from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
#from transformers import BertTokenizerFast, TFBertModel, BertConfig

max_len = 256
configuration = BertConfig()

In [5]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF Version:  2.5.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


### Setup Tokenizer

In [6]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

slow_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

save_path = "bert_base_cased"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

tokenizer = BertWordPieceTokenizer(save_path + "/vocab.txt", lowercase=False)

In [9]:
#print(tokenizer.wordpiece_tokenizer.tokenize(" ".join(sentences[0])))
#print(tokenizer.convert_tokens_to_ids(tokenizer.wordpiece_tokenizer.tokenize(" ".join(sentences[0]))))

#print(tokenizer.tokenize("I live in Germany."))
#print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I live in Germany.")))

#slow_tokenizer.convert_ids_to_tokens(16)

## Encoding(num_tokens=17, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").tokens)


['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', '.', '[SEP]']


In [7]:
!git clone https://github.com/camipower/BERT_NER_1.git

Cloning into 'BERT_NER_1'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [27]:
data_csv = "BERT_NER_1/data/ner_dataset.csv"

In [28]:
#print(df['Tag'].unique())
#print()
#print(df['Tag'].value_counts())

In [29]:
# Set a dictionary for mapping tag name to id
# tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing

tag2idx = {'O': 0,
           'X': 1,
           'B-art': 2,
           'I-art': 3,
           'B-eve': 4,
           'I-eve': 5,
           'B-geo': 6,
           'I-geo': 7,
           'B-gpe': 8,
           'I-gpe': 9,
           'B-nat': 10,
           'I-nat': 11,
           'B-org': 12,
           'I-org': 13,
           'B-per': 14,
           'I-per': 15,
           'B-tim': 16,
           'I-tim': 17,
           '[CLS]': 18,
           '[SEP]': 19}

# Map also id to 

idx2tag = {tag2idx[key]: key for key in tag2idx}

#print(tag2idx)
#print(idx2tag)


### Pre-process dataset and split into train/test sets

In [42]:
from sklearn.model_selection import train_test_split

def process_csv(data_csv):
    df = pd.read_csv(data_csv, sep=',', encoding='latin1')
    df.loc[:, 'Sentence #'] = df['Sentence #'].fillna(method='ffill')
    df['Sentence #'] = df['Sentence #'].apply(lambda x: re.sub(r"Sentence: ", "", x))
    df['Tag_id'] = df['Tag'].apply(lambda x: tag2idx[x])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tags = df.groupby("Sentence #")["Tag_id"].apply(list).values

    X_train, X_test, y_train, y_test = train_test_split(sentences, tags, train_size=0.9, random_state=42)

    #return sentences, tags    # , pos
    return X_train, X_test, y_train, y_test    # , pos


X_train, X_test, y_train, y_test = process_csv(data_csv)

In [22]:
#import random
#from sklearn.model_selection import GroupShuffleSplit
#df['Tag_id'] = df['Tag'].apply(lambda x: tag2idx[x])
#
#
#
##train = df[df['Sentence #'].astype(int).isin(train_idx)]
#
#
#
##ids = df['Sentence #'].astype(int).unique()
#np.random.seed(42)
##np.random.shuffle(ids)
##shuffled_df = df.set_index("Sentence #").iloc[ids].reset_index()
#
#groups = [df for _, df in df.groupby('Sentence #')]
#random.shuffle(groups)
#shuffled_df = pd.concat(groups).reset_index(drop=True)
#
#train_inds, test_inds = next(GroupShuffleSplit(test_size=.10, n_splits=2, random_state=42).split(shuffled_df, groups=shuffled_df['Sentence #']))
#train_df = shuffled_df.iloc[train_inds]
#test_df = shuffled_df.iloc[test_inds]


In [21]:
#train.head(10)
#train[train['Sentence #'] == "4799"]

In [23]:
#print(train.shape)
#print(test.shape)


### Create tf.data.Datasets for Training and Evaluation

In [43]:
#with tf.device('/cpu:0'):
#    train_data = tf.data.Dataset.from_tensor_slices((train_df['Word'].values, train_df['Tag_id'].values))
#    valid_data = tf.data.Dataset.from_tensor_slices((test_df['Word'].values, test_df['Tag_id'].values))
#
#    for text, label in train_data.take(1):
#        print(text)
#        print(label)

### Create input targets

In [44]:
def create_inputs_targets(sentences, tags):
    
    dataset_dict = {
        'input_ids': [],
        'token_type_ids': [],
        'attention_mask': [],
        'tags': []
    }

    for sentence, tag in zip(sentences, tags):
        input_ids = []
        target_tags = []
        for i, word in enumerate(sentence):
            ids = tokenizer.encode(word, add_special_tokens=False)
            input_ids.extend(ids.ids)
            
            ## this is   a    differentiator -> ['this', 'is', 'a', 'different', '##ia', '##tor']
            ## tag1 tag2 tag3 tag4           -> [tag1,   tag2, tag3, tag4,       tag4,   tag4]
            num_tokens = len(ids)
            target_tags.extend([tag[i]] * num_tokens)
            #target_pos.extend([pos[i]] * num_tokens)   # if we want pos too (more needs to be done)
        
        # Pad truncate
        # - 2 because 2 special tokens need to be added ([CLS], [SEP])
        input_ids = input_ids[:max_len - 2]
        target_tags = target_tags[:max_len - 2]

        input_ids = [101] + input_ids + [102]   # Add special tokens ([CLS] -> 101, [SEP] -> 102)
        #target_tags = [16] + target_tags + [16]
        target_tags = [0] + target_tags + [0]
        token_type_ids = [0] * len(input_ids)
        attention_mask = [1] * len(input_ids)
        padding_len = max_len - len(input_ids)

        input_ids = input_ids + ([0] * padding_len)
        attention_mask = attention_mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        #target_tags = target_tags + ([17] * padding_len)
        target_tags = target_tags + ([0] * padding_len)

        dataset_dict['input_ids'].append(input_ids)
        dataset_dict['token_type_ids'].append(token_type_ids)
        dataset_dict['attention_mask'].append(attention_mask)
        dataset_dict['tags'].append(target_tags)
        assert len(target_tags) == max_len, f'{len(input_ids)}, {len(target_tags)}'

    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
         dataset_dict['input_ids'],
         dataset_dict['token_type_ids'],
         dataset_dict['attention_mask']
    ]

    y = dataset_dict['tags']

    return x, y     #, tag_encoder





In [45]:
x, y = create_inputs_targets(X_train, y_train)

In [51]:
print(tokenizer.decode(x[0][0]))
print(tokenizer.encode(tokenizer.decode(x[0][0])).tokens)
print(y[0])

Kremlin officials insisted that they are cracking down on corporate crime.
['[CLS]', 'K', '##rem', '##lin', 'officials', 'insisted', 'that', 'they', 'are', 'cracking', 'down', 'on', 'corporate', 'crime', '.', '[SEP]']
[ 0 12 12 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


### Define model

In [53]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction=tf.keras.losses.Reduction.NONE
)

def masked_ce_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 17))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def create_model(num_tags):
    # BERT encoder
    encoder = TFBertModel.from_pretrained('bert-base-cased')

    # NER model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]
    embedding = layers.Dropout(0.3)(embedding)
    tag_logits = layers.Dense(num_tags + 1, activation='softmax')(embedding)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[tag_logits],
    )

    optimizer = keras.optimizers.Adam(lr=3e-5)
    
    model.compile(optimizer=optimizer, loss=masked_ce_loss, metrics=['accuracy'])

    return model


### Create model

In [54]:
#num_tags = pd.read_csv(data_csv, encoding='latin1')['Tag'].unique()
num_tags = len(tag2idx)

use_tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    use_tpu = True
except:
    use_tpu = False

if use_tpu:
    # Create distribution strategy
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model(num_tags)

else:
    model = create_model(num_tags)

model.summary()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.29.32.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.29.32.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108310272   input_1[0][0]                    
                                                                 input_3[0][0]                

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Train

In [56]:
#x_train, y_train, tag_encoder = create_inputs_targets(data_csv)
x, y = create_inputs_targets(X_train, y_train)

BATCH_SIZE = 64 if use_tpu else 16

model.fit(
    x,
    y,
    epochs=3,
    verbose=1,
    batch_size=BATCH_SIZE,
    validation_split=0.1
)

Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 256) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 256) dtype=int64>]


607/607 [==============================] - ETA: 0s - loss: 0.0420 - accuracy: 0.9895

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 256) dtype=int64>]


607/607 [==============================] - 188s 174ms/step - loss: 0.0420 - accuracy: 0.9895 - val_loss: 0.0151 - val_accuracy: 0.9950
Epoch 2/3
607/607 [==============================] - 90s 148ms/step - loss: 0.0129 - accuracy: 0.9955 - val_loss: 0.0126 - val_accuracy: 0.9955
Epoch 3/3
607/607 [==============================] - 90s 149ms/step - loss: 0.0106 - accuracy: 0.9961 - val_loss: 0.0123 - val_accuracy: 0.9955


### Tag entities

In [58]:
def create_test_input_from_text(texts):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": []
    }
    for sentence in texts:
        input_ids = []
        for idx, word in enumerate(sentence.split()):
            ids = tokenizer.encode(word, add_special_tokens=False)
            input_ids.extend(ids.ids)
            num_tokens = len(ids)
            
        # Pad and create attention masks.
        # Skip if truncation is needed
        input_ids = input_ids[:max_len - 2]

        input_ids = [101] + input_ids + [102]
        n_tokens = len(input_ids)
        token_type_ids = [0] * len(input_ids)
        attention_mask = [1] * len(input_ids)
        padding_len = max_len - len(input_ids)

        input_ids = input_ids + ([0] * padding_len)
        attention_mask = attention_mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        
        dataset_dict["input_ids"].append(input_ids)
        dataset_dict["token_type_ids"].append(token_type_ids)
        dataset_dict["attention_mask"].append(attention_mask)
        
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    return x, n_tokens



def tag_named_entities(query):

    x_test, n_tokens = create_test_input_from_text(query)

    pred_test = model.predict(x_test)
    pred_tags = np.argmax(pred_test,2)[0][1:n_tokens-1]  # ignore predictions of padding tokens

    # create dictionary of tags and their indexes
#    le_dict = dict(zip(tag_encoder.transform(tag_encoder.classes_), tag_encoder.classes_))
#    predicted_tags = [le_dict.get(_, '[pad]') for _ in pred_tags]
    predicted_tags = [idx2tag[i] for i in pred_tags]


    ids = x_test[0][0][:n_tokens]

    tokenizer_tokens = tokenizer.encode(tokenizer.decode(ids)).tokens

    # for debugging
    print(tokenizer_tokens)

    new_tokens, new_labels = [], []
    for token, label in zip(tokenizer_tokens[1:-1], predicted_tags):
        if token.startswith('##'):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_tokens.append(token)
            new_labels.append(label)
    
    for token, label in zip(new_tokens, new_labels):
        print("{}\t{}".format(token, label))

    
    
    

In [60]:
tag_named_entities(["Alex lives with Abhishek in London and works for Microsoft Corp."])


['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', 'Corp', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-org
in	O
London	B-geo
and	O
works	O
for	O
Microsoft	B-org
Corp	I-org
.	O


In [ ]:
tag_named_entities(["Alex lives with Abhishek in London and works for Ayfie."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'A', '##y', '##fie', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
for	O
Ayfie	B-per
.	O


In [ ]:
tag_named_entities(["Alex lives with Abhishek in London and works in Ayfie."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'in', 'A', '##y', '##fie', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
in	O
Ayfie	B-geo
.	O


In [61]:
tag_named_entities(["Alex lives with Abhishek in London and works for Ayfie Inc."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'A', '##y', '##fie', 'Inc', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-org
in	O
London	B-geo
and	O
works	O
for	O
Ayfie	B-org
Inc	I-org
.	I-org


In [62]:
tag_named_entities(["Alex lives with Abhishek in London and works at Microsoft's headquarters in England."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'at', 'Microsoft', "'", 's', 'headquarters', 'in', 'England', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-org
in	O
London	B-geo
and	O
works	O
at	O
Microsoft	B-org
'	O
s	O
headquarters	O
in	O
England	B-geo
.	O


In [66]:
tag_named_entities(["Abhishek and Jana are a couple."])

['[CLS]', 'A', '##b', '##his', '##he', '##k', 'and', 'Jana', 'are', 'a', 'couple', '.', '[SEP]']
Abhishek	B-per
and	O
Jana	B-per
are	O
a	O
couple	O
.	O


In [ ]:
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").tokens)
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").ids)

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', '.', '[SEP]']
[101, 3230, 2491, 1114, 138, 1830, 27516, 4638, 1377, 1107, 1498, 1105, 1759, 1111, 6998, 119, 102]


In [97]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(X_test, y_test):

    
    x, y = create_inputs_targets(X_test, y_test)
    
    model.evaluate(x, y, verbose=1)
    #pred_test = model.predict(x)
    #pred_tags = np.argmax(pred_test,2)



    #print(np.squeeze(pred_tags).shape)
    #print(np.squeeze(y).shape)

    #print(confusion_matrix(pred_tags, y))

    #print("Precision", precision_score(y, pred_tags))
    #print("Recall", recall_score(y, pred_tags))
    #print("f1_score", f1_score(y, pred_tags))

    #m = tf.keras.metrics.Precision()
    #m.update_state(y, pred_tags)
    #print(m.result().numpy())


#    result = []
#    for i in range(len(y)):
#        for sent_true, sent_pred in zip(y[i], pred_tags[i]):
#            result.append(sent_true == sent_pred)
#    print(result)


evaluate_model(X_test, y_test)


150/150 [==============================] - 6s 34ms/step - loss: nan - accuracy: 0.9957
